In [69]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy 
import string
import re
import nltk
from nltk.corpus import stopwords

In [70]:
if tf.test.is_gpu_available():
    physical_devices = tf.config.list_physical_devices('GPU')
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print('Running on GPU:', tf.test.gpu_device_name())
else:
    print('No GPU available. Running on CPU.')

No GPU available. Running on CPU.


In [71]:
x1 = pd.read_csv(r"/kaggle/input/clothesss/Ajio Fasion Clothing.csv",encoding="latin-1")
x2 = pd.read_csv(r"/kaggle/input/clothesss/myntra_products_catalog.csv")

In [72]:
x1 = x1.drop(['Brand','Id_Product','URL_image','Category_by_gender','Discount Price (in Rs.)','Original Price (in Rs.)','Color'],axis=1)

In [73]:
x2['ProductBrand'] = x2['ProductBrand'].str.replace(' ','')
x2['ProductName'] = x2['ProductName'].str.replace(' ','')
x2['Product_URL'] = 'https://www.myntra.com/' + x2['ProductBrand'] +'/' + x2['ProductName'] +'/' + x2['ProductID'].astype('str') 

In [74]:
x2 = x2.drop(['ProductID','ProductBrand','ProductName','Gender','Price (INR)','NumImages','PrimaryColor'],axis=1)

In [75]:
desc = x1['Description'].append(x2['Description'])

/tmp/ipykernel_34/1542860527.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  desc = x1['Description'].append(x2['Description'])


In [76]:
URL = x1['Product_URL'].append(x2['Product_URL'])

/tmp/ipykernel_34/924272690.py:1: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  URL = x1['Product_URL'].append(x2['Product_URL'])


In [77]:
del x1
del x2

In [62]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/5"
embed = hub.load(module_url)

In [63]:
nlp = spacy.load('en_core_web_sm')

# Download stopwords if not already downloaded
nltk.download('stopwords')

# Load stopwords
stop_words = set(stopwords.words('english'))

def preprocess_text(text_tuple, nlp_model):
    cleaned_texts = []
    batch_size = 1000  # Adjust the batch size as per your system's memory constraints

    for i in range(0, len(text_tuple), batch_size):
        batch_texts = text_tuple[i:i+batch_size]
        batch_cleaned_texts = []

        for text in batch_texts:
            # Convert text to lowercase
            text = text.lower()

            # Remove special characters and symbols
            text = re.sub(r"[^a-zA-Z0-9\s]", "", text)

            # Remove punctuation
            text = text.translate(str.maketrans("", "", string.punctuation))

            batch_cleaned_texts.append(text)

        # Process the batch of cleaned texts using spaCy
        doc_batch = nlp_model.pipe(batch_cleaned_texts, batch_size=batch_size)
        
        for doc in doc_batch:
            words = [token.text for token in doc if token.text not in stop_words and token.is_alpha]

            # Join the words back into a single string
            cleaned_text = " ".join(words)
            cleaned_texts.append(cleaned_text)

    return list(cleaned_texts)

desc = preprocess_text(desc, nlp)

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [64]:
import re

# Preprocessing function to tokenize the input text
def preprocess_intext(text):
    # Convert text to lowercase
    text = text.lower()
    # Tokenize by splitting on whitespace and non-alphanumeric characters
    tokens = re.findall(r'\w+', text)
    return tokens


In [110]:
def compute_similarity(descriptions, urls, input_text, top_n=5):
    # Preprocess the input text
    preprocessed_input = preprocess_intext(input_text)
    preprocessed_input = [word for word in preprocessed_input if len(word) > 1] 
    # Preprocess the descriptions
    preprocessed_descriptions = desc
    
    print("Preprocessed input:", preprocessed_input)
#     print("Preprocessed descriptions:", preprocessed_descriptions)
 

    # Vectorize the preprocessed input and descriptions using TF-IDF
    vectorizer = TfidfVectorizer()
    input_vectors = vectorizer.fit_transform([' '.join(preprocessed_input)])
    description_vectors = vectorizer.transform(preprocessed_descriptions)
     
    # Compute the cosine similarity between the input and description vectors
    similarity_scores = cosine_similarity(input_vectors, description_vectors)[0]

    # Get indices of top-N similar items
    ranked_indices = np.argsort(similarity_scores)[::-1][:top_n]

    # Get the top-N URLs 
    top_n_urls = URL.iloc[ranked_indices].values

    return top_n_urls


In [115]:

top_n_urls = compute_similarity(desc, URL, input_text='Yellow top ', top_n=5)

for url in zip(top_n_urls):
    print("URL:", url)
    print()

Preprocessed input: ['yellow', 'top']
URL: ('https://www.myntra.com/ParkAvenue/ParkAvenueWomenYellowSolidTop/10004401',)

URL: ('https://www.ajio.com/outryt-women-striped-round-neck-boxy-sleeves-mustard-yellow-top/p/460546941_mustard',)

URL: ('https://www.myntra.com/DKNY/DKNYWomenNectarLoveEaudeParfum100ml/10266631',)

URL: ('https://www.myntra.com/VanHeusenWoman/VanHeusenWomanYellowSelfDesignTop/10206161',)

URL: ('https://www.myntra.com/GlobalDesi/GlobalDesiWomenMustardYellow&MaroonPrintedRegularTop/10247005',)

